In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [2]:
df = pd.read_csv("data/parenting_dataset_asli.csv", index_col="NO")
df.drop(columns=["Responden"], inplace=True)
df.head()

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,Parenting_style
NO,,,,,,,,,,,,,,,,,,,,,
1,1,5,2,5,1,4,1,4,1,5,...,4,3,1,4,1,2,4,1,1,Authoration
2,1,4,2,4,2,5,1,4,5,5,...,2,3,2,3,4,3,2,2,1,Authorative
3,2,4,4,5,5,4,2,5,5,5,...,4,3,4,4,4,4,4,3,3,Authorative
4,1,3,2,4,2,4,1,3,2,5,...,5,2,4,3,3,3,5,5,4,Permissive
5,1,1,3,3,1,1,1,5,3,3,...,5,1,4,3,4,3,4,4,3,Permissive


In [3]:
df.Parenting_style.value_counts()

Authorative    182
Authoration    163
Permissive     155
Name: Parenting_style, dtype: int64

In [4]:
X = df.drop(columns="Parenting_style")
y = df.Parenting_style

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 52), (100, 52), (400,), (100,))

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from jcopml.tuning import grid_search_params as gsp

In [6]:
 gsp.

{'algo__n_estimators': [100, 150, 200],
 'algo__max_depth': [20, 50, 80],
 'algo__max_features': [0.3, 0.6, 0.8],
 'algo__min_samples_leaf': [1, 5, 10]}

In [11]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(scaling='minmax'), X_train.columns),
])

pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', DecisionTreeClassifier(n_jobs=-1, random_state=42))
])

model = GridSearchCV(pipeline, gsp.rf_params, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [20]:
save_model(model.best_estimator_, "pakis_model.h5")

Model is pickled as model/pakis_model.h5
